In [11]:
%pip install -qU langchain-community langgraph langchain-anthropic tavily-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [5]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718377791, 'localtime': '2024-06-14 8:09'}, 'current': {'last_updated_epoch': 1718377200, 'last_updated': '2024-06-14 08:00', 'temp_c': 12.5, 'temp_f': 54.5, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 5.1, 'wind_kph': 8.3, 'wind_degree': 206, 'wind_dir': 'SSW', 'pressure_mb': 1017.0, 'pressure_in': 30.04, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 77, 'cloud': 0, 'feelslike_c': 11.8, 'feelslike_f': 53.2, 'windchill_c': 11.8, 'windchill_f': 53.2, 'heatindex_c': 12.5, 'heatindex_f': 54.5, 'dewpoint_c': 8.4, 'dewpoint_f': 47.1, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 4.0, 'gust_mph': 6.8, 'gust_kph': 11.0}}"}, {'url': 'https://www.accuweather.com/en/us/san-francisco/9

In [9]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [15]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

 ········


In [17]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [19]:
model_with_tools = model.bind_tools(tools)

In [21]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [23]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_xwOqA68HnHPhkm4HXkHTMHyi'}]


In [25]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [27]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='348679ff-ca35-4e20-9ef3-5c9dbbd823b4'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-60bb9090-48d1-425f-b0a9-f79ee0395184-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91})]

In [29]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='2abedb8b-ffb6-4213-8aaf-2993c7cd34d0'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_h8uC2zoIpmki581GI1xMgpqj', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-04064a6a-11d8-4d67-8ae6-615913cd4b7f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_h8uC2zoIpmki581GI1xMgpqj'}], usage_metadata={'input_tokens': 86, 'output_tokens': 22, 'total_tokens': 108}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of Am

In [31]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ebWVQU4qRRvaC3EXx4BQUDTZ', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3291af39-0ec1-457a-907a-e69e4935aaa7-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_ebWVQU4qRRvaC3EXx4BQUDTZ'}], usage_metadata={'input_tokens': 86, 'output_tokens': 22, 'total_tokens': 108})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42

In [33]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

/Users/musabello/anaconda3/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718378681, 'localtime': '2024-06-14 8:24'}, 'current': {'last_updated_epoch': 1718378100, 'last_updated': '2024-06-14 08:15', 'temp_c': 12.5, 'temp_f': 54.5, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 5.1, 'wind_kph': 8.3, 'wind_degree': 206, 'wind_dir': 'SSW', 'pressure_mb': 1017.0, 'pressure_in': 30.04, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 77, 'cloud': 0, 'feelslike_c': 11.8, 'feelslike_f': 53.2, 'windchill_c': 11.8, 'windchill_f': 53.2, 'heatindex_c': 12.5, 'heatindex_f': 54.5, 'dewpoint_c': 8.4, 'de

In [35]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [37]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [39]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-42217982-1478-4ecf-bb56-31c0c49014c3-0', usage_metadata={'input_tokens': 83, 'output_tokens': 11, 'total_tokens': 94})]}}
----


In [45]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who is Bello Musa also known as Musbell?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qSXlnr9hPDC7eTdiWt0TgZFr', 'function': {'arguments': '{"query":"Bello Musa also known as Musbell"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 138, 'total_tokens': 163}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_aa87380ac5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c904294a-4566-4f30-ae16-0a6ff8b9584c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Bello Musa also known as Musbell'}, 'id': 'call_qSXlnr9hPDC7eTdiWt0TgZFr'}], usage_metadata={'input_tokens': 138, 'output_tokens': 25, 'total_tokens': 163})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://ma.linkedin.com/in/musbell", "content": "Frontend Engineer at FlexiSAF Edusoft Limited \\u00b7 Hi, I&#39;m Bello Musa, a Full-Stack Engineer.I have over 5 years of hands-on experie